In [15]:
## 기본 모듈
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

## 딥러닝 모듈
import tensorflow
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, concatenate, AveragePooling2D
from keras.layers import BatchNormalization, Dropout
from keras.utils import to_categorical
from keras.losses import MSE
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

In [3]:
from keras.datasets import cifar10

## 01. cifar10 Function

In [34]:
from keras.applications import VGG16, VGG19

In [35]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [36]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [37]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 10)
(10000, 32, 32, 3) (10000, 10)


In [38]:
base1_model = VGG19(include_top=False,
                    input_shape=(32,32,3))

In [49]:
### conv, pooling
conv1_1 = Conv2D(50,(3,3),activation="relu")
conv1_2 = Conv2D(100,(3,3),activation="tanh")
pooling1_1 = MaxPooling2D((3,3))
conv1_3 = Conv2D(150,(2,2),activation="relu")
pooling1_2 = MaxPooling2D((2,2))

conv2_1 = Conv2D(30,(2,2),activation="relu")
conv2_2 = Conv2D(50,(2,2),activation="relu")
pooling2_1 = AveragePooling2D((2,2))
pooling2_2 = AveragePooling2D((2,2))

flatten = Flatten()

affine1 = Dense(128,activation="relu")
output = Dense(10,activation="softmax")

In [50]:
input_shape = X_train.shape[1:]
input_1 = Input(shape=input_shape)
input_2 = Input(shape=input_shape)

In [51]:
# 왼쪽
conv_11 = conv1_1(input_1)
conv_12 = conv1_2(conv_11)
conv_13 = pooling1_1(conv_12)
conv_14 = conv1_3(conv_13)
conv_15 = pooling1_2(conv_14)

# 오른쪽
conv_21 = conv2_1(input_2)
conv_22 = conv2_2(conv_21)
conv_23 = pooling2_1(conv_22)
conv_24 = pooling2_2(conv_23)

# faltten
flatten1 = flatten(conv_15)
flatten2 = flatten(conv_24)

# concat
concat = concatenate([flatten1, flatten2])

# layers
l1 = affine1(concat)
output = output(l1)

# model
model = Model(inputs=[input_1,input_2], outputs=[output])

In [52]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_31 (Conv2D)             (None, 30, 30, 50)   1400        ['input_12[0][0]']               
                                                                                                  
 input_13 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_32 (Conv2D)             (None, 28, 28, 100)  45100       ['conv2d_31[0][0]']              
                                                                                              

In [53]:
from keras.utils import plot_model

In [54]:
plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [55]:
plot_model(model,to_file="model.png")

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [56]:
X_train.shape

(50000, 32, 32, 3)

In [61]:
model.compile(optimizer="rmsprop",
                loss = "categorical_crossentropy",
                metrics=["accuracy"])

In [68]:
model.fit(x = (X_train, X_train),
            y = y_train,
            epochs=20,
            batch_size=1000,
            validation_data=([X_test,X_test],y_test))

Epoch 1/20


KeyboardInterrupt: 